<a href="https://colab.research.google.com/github/jooeun921/Big-Data-Analyst/blob/main/Part02_Section_03_data_preprocessing_scikit_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

### Section 03 학습 : scikit-learn을 활용한 데이터 전처리

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/dat.csv')

print(df.info())

In [ ]:
df.head()

In [ ]:
# X는 grade를 제외한 나머지 열. y는 grade => 종속변수이자, 반응변수

X = df.drop(['grade'], axis=1)
y  = df.grade

#### 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, shuffle = True, stratify = None)

In [ ]:
# 학교 종류가 GP, MS 으로 나눠져 있으므로, stratify(층화추출)을 활용하여 각 학교 종류에 따른 동일한 비율로 train, test 데이터 분할. 비슷한 맥락으로, 성별로도 가능.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify = X['school'])

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(nrows = 1, ncols = 2)

y_train.hist(ax = axs[0], color = 'blue', alpha = 0.7)
axs[0].set_title('histogram of train y')

y_test.hist(ax = axs[1], color = 'red', alpha = 0.7)
axs[1].set_title('histogram of test y')

plt.tight_layout();
plt.show()

#### 결측치 처리
대치법 종류에는 대표값(평균, 중앙값, 최빈값)을 이용하는 방법과 KNN을 이용한 대치법이 있음.

대치법을 활용할 때, 결측치를 대치할 값을 선정한 뒤에 실제로 교체하는 과정에서, `fit()`과 `transform()` 함수를 사용하게 된다.

`fit()` : 데이터(평균, 중앙값 등)를 학습.
`transform()` : 학습된 데이터를 활용해서 데이터를 변환.

`fit_transform()` : 한번에 적용할 수 있음.

🍎 단, 이는 학습 데이터에서만 활용되고, 테스트 데이터에서는 `transform()`만 사용하는데, 학습 시에 활용한 기준으로만 대치해야 하고. 테스트 데이터에서 학습시킨 걸 활용하면 데이터 누출이 생긴다고 보기 때문!


```
# 학습시킬 데이터 = imputer
# 변환할 데이터 = X_train or X_test

X_train = imputer.fit_transform(X_train)
```

In [ ]:
print(df.isna().sum())

In [ ]:
# 평균 대치법
# 쉽고 빠르게 대치할 수 있지만. 다른 변수 간의 상관관계를 고려하지 못하고, 결측치가 많으면 평균값의 빈도수가 많아지므로 분포가 왜곡된다.

from sklearn.impute import SimpleImputer

train_X1 = X_train.copy()
test_X1 = X_test.copy()
imputer_mean = SimpleImputer(strategy = 'mean')

train_X1['goout'] = imputer_mean.fit_transform(train_X1[['goout']])
test_X1['goout'] = imputer_mean.transform(test_X1[['goout']])

print(test_X1['goout'].isna().sum())
print(test_X1['goout'].isna().sum())

In [ ]:
# 중앙값 대치법
# 쉽고 빠르게 대치할 수 있지만. 다른 변수 간의 관계를 고려하지 못함. 결측치가 많을 때 중앙값의 빈도수가 많아지므로 분포가 왜곡.

from sklearn.impute import SimpleImputer

train_X2 = X_train.copy()
test_X2 = X_test.copy()
imputer_median = SimpleImputer(strategy = 'median')

train_X2['goout'] = imputer_median.fit_transform(train_X2[['goout']])
test_X2['goout'] = imputer_median.transform(test_X2[['goout']])

print(train_X2['goout'].isna().sum())
print(test_X2['goout'].isna().sum())

In [ ]:
# 최빈값 대치법
# 쉽고 빠르게 대치할 수 있지만. 다른 변수 간의 상관관계를 고려하지 못함. 범주형 변수 결측치 대치에 활용할 경우 분포 불균형을 심화시킬 수 있음.

from sklearn.impute import SimpleImputer

train_X3 = X_train.copy()
test_X3 = X_test.copy()
imputer_frequent = SimpleImputer(strategy = 'most_frequent')

train_X3['goout'] = imputer_frequent.fit_transform(train_X3[['goout']])
test_X3['goout'] = imputer_frequent.transform(test_X3[['goout']])

print(train_X3['goout'].isna().sum())
print(test_X3['goout'].isna().sum())

In [ ]:
# KNN(K-Nearest Neighbors)을 이용한 대치법
# 데이터에 대한 가정 없이 쉽고 빠르게 결측치 대치 가능
# 변수 스케일 및 이상치에 민감하며, 고차원 데이터의 경우 모델 성능이 떨어질 수 있음.

from sklearn.impute import KNNImputer

train_X4 = X_train.copy()
test_X4 = X_test.copy()

# KNN은 결측치 샘플에 대해서 이웃 간의 거리를 기반으로 계산하기 때문에, 범주형 데이터에는 활용할 수 없음. -> 숫자형만 따로 뽑은 뒤에 대치해줘야 함.
# 여기서부터 train_X4_num과 test_X4_num은 numpy.ndarray 타입으로 되어 있기 때문에,
train_X4_num = train_X4.select_dtypes('number')
test_X4_num = test_X4.select_dtypes('number')

train_X4_cat = train_X4.select_dtypes('object')
test_X4_cat = test_X4.select_dtypes('object')

knnimputer = KNNImputer(n_neighbors  = 3)
train_X4_num_imputed = knnimputer.fit_transform(train_X4_num)
test_X4_num_imputed = knnimputer.transform(test_X4_num)

train_X4_num_imputed = pd.DataFrame(train_X4_num_imputed, columns = train_X4_num.columns, index = train_X4.index)
test_X4_num_imputed = pd.DataFrame(test_X4_num_imputed, columns = test_X4_num.columns, index = test_X4.index)

# axis = 1 -> columns 기준으로, 수치형 데이터(대치함)와 범주형 데이터 병합.
train_X4 = pd.concat([train_X4_cat, train_X4_num_imputed], axis = 1)
test_X4 = pd.concat([test_X4_cat, test_X4_num_imputed], axis = 1)

print(train_X4['goout'].isna().sum())
print(test_X4['goout'].isna().sum())

보통의 Imputer(SimpleImputer, KNNImputer, StandardScaler 등)은 사용 시 데이터를 numpy.ndarray 형태로 반환한다.
이때, Imputer를 정의할 때 `.set_output(transform = 'pandas')` 를 붙이면 자동으로 DataFrame 타입으로 반환해줌.

In [ ]:
knnimputer2 = KNNImputer(n_neighbors = 5).set_output(transform = 'pandas')
train_X4_num_imputed2 = knnimputer2.fit_transform(train_X4_num)
test_X4_num_imputed2 = knnimputer2.transform(test_X4_num)

print(train_X4_num_imputed2.head())

In [ ]:
# pandas를 활용한 간단한 결측치 처리하기

data = {
    '학생': ['철수', '영희', '민수', '수지', '지현'],
    '수학': [85, np.nan, 78, np.nan, 93],
    '영어': [np.nan, 88, 79, 85, np.nan],
    '과학': [92, 85, np.nan, 80, 88]
}

df = pd.DataFrame(data)

In [ ]:
# 평균값으로 결측치 채우기
# inplace는 기존의 데이터프레임에서 값을 변경할지, 아니면 새로운 데이터프레임을 만들지에 관한 파라미터임.
# False를 주는 경우, 새로운 변수에 할당을 해줘야, 기존 데이터프레임에는 변화가 없고 새 데이터프레임에만 결측치가 채워진 결과가 나옴.

df1 = df.copy()

# 근데 python에서 inplace를 사용한 방식이 점점 줄어들다가 사라지는 추세라, 원본을 수정하려면 직접 할당하는 것이 낫다고 경고문 뜸.
# df1['수학'].fillna(df1['수학'].mean(), inplace = True)
# df1['영어'].fillna(50, inplace = True)
# df1['과학'].fillna(method = 'ffill',  inplace = True)

df1['수학'] = df1['수학'].fillna(df1['수학'].mean())
df1['영어'] = df1['영어'].fillna(50)
df1['과학'] = df1['과학'].ffill()

print(df1)

In [ ]:
df2 = df.copy()

# apply 함수를 적용하기 위해서 수치형 변수만을 따로 뽑음. -> df2_num이라는 새로운 DataFrame에 저장.
df2_num = df2.select_dtypes('number')

# apply 함수와 lambda를 사용하여 np.nan 결측치를 그 칼럼(열)의 평균값으로 채움. df2_num은 결측치가 전부 채워져있고, 수치형 데이터만 존재함.
df2_num = df2_num.apply(lambda col : col.fillna(col.mean()))

# 기존의 df2 DataFrame에서 수치형 데이터가 든 열만 뽑아서, df2_num(결측치가 없는)의 값으로 변경.
df2[df2_num.columns] = df2_num

print(df2)

#### 범주형 변수 처리
범주형 변수를 컴퓨터가 이해할 수 있는 수치형으로 변환하는 것. 대표적인 변수 인코딩 방식으로는, Ladel encoding, One-hot encoding, Dummy encoding 등이 있다.

🍎 범주형 변수 축소(Class coercing)

그러나, 범주형 설명변수(ex. 지역) 같은 경우 범주의 수가 많거나 불균형인 경우에 원-핫 인코딩을 하는 경우 칼럼의 수가 늘어나 모델 학습 시간이 오래 걸릴 수 있음. -> 범주 간의 통합이 필요한 이유.


In [ ]:
# Label encoding
# 범주형 변수의 label에 알파벳 순서대로 고유한 정수를 할당.
# 순서형 변수의 경우, 순서를 반영한 인코딩 가능.
# 순서가 없는 변수의 경우, 잘못된 순서 정보가 반영되는 문제 발생.

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

train_X5 = X_train.copy()
test_X5 = X_test.copy()

train_X5_cat = train_X5.select_dtypes('object')
test_X5_cat = test_X5.select_dtypes('object')

In [ ]:
# LabelEncoder는 1차원. 하나의 열에 대해서만 인코딩 가능하고,
# OrdinalEncoder 같은 경우에는 2차원. 여러열을 동시에 인코딩할 수 있기 때문에, 보통 OrdinalEncoder를 사용함.

# labelencoder = LabelEncoder().set_output(transform = 'pandas')
ordinalencoder = OrdinalEncoder().set_output(transform = 'pandas')
train_X5_cat = ordinalencoder.fit_transform(train_X5_cat)
test_X5_cat = ordinalencoder.transform(test_X5_cat)

print(train_X5_cat.head())

In [ ]:
# train-test 인코딩을 할 때, test data는 train에서 학습한 범주에 대해서만 범주 변경을 하므로,
# train_data에서 존재하지 않는 값이 test data에서 주어지게 되면 ValueError가 발생함.
# 따라서 이를 막기 위해, OrdinalEncoder를 설정할 때 예외처리를 해줘야 한다.

train_data = pd.DataFrame( { 'job' : ['Doctor', 'Engineer', 'Teacher', 'Nurse'] } )
test_data = pd.DataFrame( { 'job' : ['Doctor', 'Lawyer', 'Teacher', 'Scientist'] } )

# handle_unknown : {'error', 'use_encoded_value'}, default='error'
'''
When set to
        'use_encoded_value', the encoded value of unknown categories will be
        set to the value given for the parameter `unknown_value`.
'''

oe = OrdinalEncoder(handle_unknown = 'use_encoded_value', unknown_value = -1)

train_data['job_encoded'] = oe.fit_transform(train_data[['job']])
test_data['job_encoded'] = oe.transform(test_data[['job']])

print(train_data)
print(test_data)

In [ ]:
# One-hot encoding
# Label encoding의 범주형 변수에 순서 정보가 반영되는 문제 해결 가능
# 범주형 변수가 많을 경우, 차원이 늘어남에 따라 계산량이 늘어나는 문제 발생.

from sklearn.preprocessing import OneHotEncoder

train_X6 = X_train.copy()
test_X6 = X_test.copy()

train_X6_cat = train_X6.select_dtypes('object')
test_X6_cat = test_X6.select_dtypes('object')

In [ ]:
# sparse_output는 희소행렬 / 일반적인 밀집행렬 선택 옵션임. 사람이 보기 편한 쪽은 밀집행렬이기 때문에 따로 시각화용도면 False가 맞지만,
# 그냥 데이터 전처리 후에 모델 돌리는 목적으로 인코딩을 하는 거라면 굳이, sparse_output 을 False로 할 필요가 없음.
onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown  = 'ignore').set_output(transform = 'pandas')

train_X6_cat = onehotencoder.fit_transform(train_X6_cat)
test_X6_cat = onehotencoder.transform(test_X6_cat)

print(train_X6_cat.head())

In [ ]:
# Dummy encoding
# 범주형 변 수의 각 범주 수 -1개 만큼의 새로운 열을 샌성하고, 1 또는 0의 값을 부여해서 각 범주를 구분하는 방법.
# 기존의 원-핫인코딩에서 칼럼 1개를 제거하면 더미 인코딩이 됨.
# 장점 :  Label encoding의 문제점인 순서 정보가 반영되는 문제를 해결 가능
# 단점 : One-hot encoding과 마찬가지로 범주형 변수가 많을 경우 차원이 늘어남에 따라 계산량이 늘어남.

train_X7 = X_train.copy()
test_X7 = X_test.copy()

train_X7_cat = train_X7.select_dtypes('object')
test_X7_cat = test_X7.select_dtypes('object')

oe = OneHotEncoder(drop = 'first', sparse_output = False, handle_unknown='error').set_output(transform = 'pandas')

train_X7_cat = oe.fit_transform(train_X7_cat)
test_X7_cat = oe.transform(test_X7_cat)

print(train_X7_cat.head())

In [ ]:
# 기존 수치형 컬럼만 따로 보존
train_X7_num = train_X7.select_dtypes(exclude='object')
test_X7_num = test_X7.select_dtypes(exclude='object')

# 수치형 + 인코딩된 범주형 병합
train_X7 = pd.concat([train_X7_num, train_X7_cat], axis=1)
test_X7 = pd.concat([test_X7_num, test_X7_cat], axis=1)

print(train_X7.head())

In [ ]:
# 범주형 변수 범주 축소 방법

train_bike = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/bike_train.csv')
test_bike = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/bike_test.csv')
print(train_bike.head(2))

In [ ]:
print(train_bike['weather'].value_counts())

In [ ]:
# 각 범주 간의 상대적 빈도로 계산.
freq = train_bike['weather'].value_counts(normalize = True)
print(freq)

In [ ]:
# 여기에서 weather = 3, 4는 데이터적으로 큰 의미를 가지지 못하기 때문에, 범주를 통합하는 게 나음.
# 해당

rare_categories = freq[freq < 0.1].index
print(rare_categories)

In [ ]:
train_bike['weather'] = train_bike['weather'].mask(train_bike['weather'].isin(rare_categories), 'other')
test_bike['weather'] = test_bike['weather'].mask(test_bike['weather'].isin(rare_categories), 'other')

print(train_bike['weather'].value_counts())

#### 변수 변환 및 스케일링
데이터 분포의 치우침이 있을 때, 변수 변환을 통해 정규분포 형태로 변환하는 것.
- 일반 변환 : Box-Cox 변환, Yeo-Johnson 변환, 로그 변환, 루트 변환.
- 정규화 방법 : 표준화(Standardization), min-max 등

In [ ]:
from sklearn.preprocessing import PowerTransformer
import warnings
import matplotlib.pyplot as plt

np.warnings = warnings

bike_data = pd.read_csv("https://raw.githubusercontent.com/YoungjinBD/data/main/bike_train.csv")
bike_data['count'].hist()
plt.show()

In [ ]:
box_tr = PowerTransformer(method = 'box-cox') # method = 'yeo-johnson'이 default 값.
bike_data['count_boxcox'] = box_tr.fit_transform(bike_data[['count']])

print("lambda: ", box_tr.lambdas_)

In [ ]:
bike_data['count_log'] = np.log1p(bike_data[['count']])
bike_data['count_sqrt'] = np.sqrt(bike_data[['count']])

bike_data[['count', 'count_boxcox', 'count_log', 'count_sqrt']].hist()
plt.show()

In [ ]:
# 정규화 방법

from sklearn.preprocessing import StandardScaler

train_X8 = X_train.copy()
test_X8 = X_test.copy()

standardscaler = StandardScaler().set_output(transform = 'pandas')

train_X8_num = train_X8.select_dtypes('number')
test_X8_num = test_X8.select_dtypes('number')

train_X8_num = standardscaler.fit_transform(train_X8_num)
test_X8_num = standardscaler.transform(test_X8_num)

fig, axs = plt.subplots(nrows = 1, ncols = 2)
train_X8['absences'].hist(ax = axs[0], color = 'blue', alpha = 0.7)
train_X8_num['absences'].hist(ax = axs[1], color = 'red', alpha = 0.7)
axs[0].set_title('before')
axs[1].set_title('after')

plt.tight_layout()
plt.show()

In [ ]:
print(np.round(train_X8['absences'].mean()))
print(np.round(train_X8_num['absences'].mean()))
print(np.round(train_X8['absences'].std(), 2))
print(np.round(train_X8_num['absences'].std(), 2))

행 : axis = 0

열 : axis = 1

이게 기본 베이스긴 하지만, 실제 계산할 때는 axis = 0일 때 열 기준의 합을 구하게 됨. 즉, columns 별로 합을 구할 때는 axis = 0임.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

train_X9 = X_train.copy()
test_X9 = X_test.copy()

train_X9_num = train_X9.select_dtypes('number')
test_X9_num = test_X9.select_dtypes('number')

minmaxscaler = MinMaxScaler().set_output(transform = 'pandas')

train_X9_num = minmaxscaler.fit_transform(train_X9_num)
test_X9_num = minmaxscaler.fit_transform(test_X9_num)

range_df = train_X9_num.select_dtypes('number').apply(lambda x: x.max() - x.min(), axis = 0)
print(range_df)

In [ ]:
# 이상치 처리

warpbreaks = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/warpbreaks.csv')
warpbreaks.boxplot(column = ['breaks']);
plt.show();

In [ ]:
# IQR를 활용한 이상치 처리

Q1 = np.quantile(warpbreaks['breaks'], 0.25)
Q3 = np.quantile(warpbreaks['breaks'], 0.75)

IQR = Q3 - Q1

UC = Q3 + ( 1.5 * IQR )
LC = Q1 - ( 1.5 * IQR )

print(warpbreaks.loc[(warpbreaks.breaks > UC) | (warpbreaks.breaks < LC), :])

In [ ]:
# Z-score를 이용한 방법 ( 평균을 중심으로 표준편차 대비 몇배 떨어져 있는지를 기준으로 함.)
# Z = (x-평균)/표준편차. 보통 Z > 3 정도로 이상치 판별기준을 세움.

upper = warpbreaks['breaks'].mean() + ( 3 * warpbreaks['breaks'].std())
lower = warpbreaks['breaks'].mean() - ( 3 * warpbreaks['breaks'].std())

warpbreaks.loc[(warpbreaks.breaks > upper) | (warpbreaks.breaks < lower), :]

In [ ]:
# 이산화 방법
# 수치형 변수를 그룹핑하여 범주형 변수로 변환하는 것.

from sklearn.preprocessing import KBinsDiscretizer

# .T = 전치연산. (1, 9) 배열을 (9, 1) 배열로 변경.
X = np.array( [[0, 1, 1, 2, 5, 10, 11, 14, 18]] ).T

In [ ]:
# 구간 간의 길이가 맞도록.
kbd = KBinsDiscretizer(n_bins = 3, strategy = 'uniform')

X_bin = kbd.fit_transform(X).toarray()
print(kbd.bin_edges_)

In [ ]:
# 분위수를 기준으로 구간 분리
kbd2 = KBinsDiscretizer(n_bins = 3, strategy = 'quantile')

X_bin2 = kbd2.fit_transform(X).toarray()
print(kbd2.bin_edges_)

In [ ]:
# 구간을 임의로 설정하는 방법.
# pd.cut()은 연속형 숫자 데이터를 구간(bins) 으로 나누고, 각 데이터가 어느 구간에 속하는지 라벨(labels) 을 붙여주는 함수임.

bins = [0, 4, 7, 11, 18]
labels = ['A', 'B', 'C', 'D']

# X 배열에 대해서 bins는 구간 설정. 참고로 구간은 앞은 열림, 뒤는 닫힘임. (0, 4] = 0 < x <= 4
# 해당 구간에 대한 라벨은 ladels로 설정.

# reshape(-1)은 1차원 행렬로 평탄화하는 것.
X_bin3 = pd.cut(X.reshape(-1), bins = bins, labels = labels)
print(X_bin3)

#### 차원축소
- 데이터 희소성 : 고차원 공간에서는 공간 내에 데이터가 희소해지므로, 데이터 간의 거리가 - 멀어지고 의미있는 패턴을 찾기 어려워짐.
- 계산 복잡성 증가 : 차원이 증가할수록 계산해야 할 양이 기하급수적으로 증가.
- 과적합 : 모델의 일반화 성능 감소.

이를 해결하기 위해 차원축소를 진행해야 하는데, 대표적인 방법으로는 `주성분 분석(PCA)`이 있음.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/df_5.csv')
df.columns = ['index', 'X100m', 'Long.jump', 'Shot.put', 'High.jump', 'X400m', 'X110m.hurdle', 'Discus', 'Pole.vault', 'Javeline', 'X1500m']
df.set_index('index', inplace = True)

print(df.head( ))
print(df.shape)

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, random_state=42)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = train.corr()
sns.heatmap(corr_matrix, annot = True, cmap = "coolwarm", fmt = ".2f")
plt.title("Correlation Matrix (Train Data)")
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

standardscaler = StandardScaler()

train_scaler = standardscaler.fit_transform(train)
test_scaler = standardscaler.transform(test)

In [ ]:
from sklearn.decomposition import PCA

# 전체 변수에 대한 주성분 분석을 진행한 뒤에 np.cumsum(pca.explained_variance_ratio_) 등을 활용하여 직접 주성분 개수를 선택해도 되고,
pca = PCA(n_components=10, svd_solver = 'auto')
train_scaler_pca = pca.fit_transform(train_scaler)
test_scaler_pca = pca.transform(test_scaler)

In [ ]:
import matplotlib.pyplot as plt

cumulative_explained_varience = np.cumsum(pca.explained_variance_ratio_ )

print(np.cumsum(pca.explained_variance_ratio_))

# plt.figure(figsize = (8, 6))
# plt.plot(range(1, len(cumulative_explained_varience) + 1), cumulative_explained_varience, marker = 'o', linestyle = '-')
# plt.xlabel('Number of Principal Components')
# plt.ylabel('Cumulative Explained Variance')
# plt.title('Scree Plot')
# plt.grid(True)
# plt.show()

In [ ]:
# 아에 누적 분산 비율을 임의로 설정하여, 주성분의 수를 자동으로 정하게 할 수도 있음.

pca2 = PCA(n_components=0.8, svd_solver = 'full')
train_scaler_pca2 = pca2.fit_transform(train_scaler)
test_scaler_pca2 = pca2.transform(test_scaler)

print(pca2.explained_variance_ratio_)
print(pca2.n_components_)

#### 범주형 변수와 연속형 변수 한번에 전처리하기
기존에는 범주형과 연속형을 나누고, 개별적으로 전처리한 뒤에 병합하는 형태로 진행했다면, `make_column_transformer()`를 사용하면, 한번에 처리할 수 있음.

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split


dat = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/bda1.csv')

y = dat.grade
X = dat.drop(['grade'], axis = 1)

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# make_column_transformer 함수 활용해서 한번에 처리하기
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

cat_columns = train_X.select_dtypes('object').columns
num_columns = train_X.select_dtypes('number').columns

onehotencoder = OneHotEncoder(sparse_output = False, drop = None, handle_unknown='ignore')
stdscaler = StandardScaler()

# 여기에서 remainder = 'passthrough'는 지정하지 않는 column에 대해서 어떻게 작업할지에 대한 파라미터임. default 값은 drop이라, 아에 삭제시켜버림.
mc_transformer = make_column_transformer((stdscaler, num_columns), (onehotencoder, cat_columns), remainder = 'passthrough').set_output(transform = 'pandas')

train_X_transformed = mc_transformer.fit_transform(train_X)
test_X_transformed = mc_transformer.transform(test_X)

print(train_X_transformed.head(2))

In [ ]:
# ColumnTransformer는 make_column_transformer 와 완전히 동일한 역할을 수행함. => 연속형, 범주형 데이터 한번에 처리할 수 있음.

from sklearn.compose import ColumnTransformer

onehotencoder = OneHotEncoder(sparse_output = False, drop = None, handle_unknown='ignore')
stdscaler = StandardScaler()

c_transformer = ColumnTransformer([
    ('cat', onehotencoder, cat_columns), ('num', stdscaler, num_columns)
]).set_output(transform = 'pandas')

train_X2_transformed = c_transformer.fit_transform(train_X)
test_X2_transformed = c_transformer.transform(test_X)

print(train_X2_transformed.head(2))

In [ ]:
# 데이터 누수
# 학습 데이터를 기반으로 테스트 데이터(미래)를 예측해야 하는 것인데, fit_transform 등으로 테스트 데이터까지도 학습을 진행하여 반영하게 되면 데이터 누수가 발생하게 된다.
from sklearn.preprocessing import StandardScaler

trainX = pd.DataFrame({'x1': range(1, 9, 1), 'x2': range(15, 23, 1)})
testX = pd.DataFrame({'x1': [1, 3, 5], 'x2': [2, 4, 6]})

stdscaler = StandardScaler( )
trainX2 = stdscaler.fit_transform(trainX)
testX2 = stdscaler.transform(testX)

print('trainX 1의 표준화 변환 결과 :', trainX2[0, 0])
print('testX 1의 표준화 변환 결과 :', testX2[0, 0])

### Section 03 연습문제 : scikit-learn을 활용한 데이터 전처리

In [ ]:
import pandas as pd
import numpy as np

##### 1-3번

In [ ]:
#1 다음 데이터에서 'age' 변수의 결측치를 중앙값으로 대치한 후, 'age' 변수의 평균을 계산하여 출력하시오.

data = {
'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
'age': [25, np.nan, 30, np.nan, 22],
'score': [85, 90, 78, 88, 95]
}

df = pd.DataFrame(data)
df['age'] = df['age'].fillna(df['age'].median())
print(df['age'].mean())

In [ ]:
#2 다음 DataFrame에서 'city' 변수를 원-핫 인코딩(one-hot encoding)하여 새로운 DataFrame으로 출력하시오.
from sklearn.preprocessing import OneHotEncoder

data = pd.DataFrame({
'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
'city': ['Seoul', 'Busan', 'Seoul', 'Daegu', 'Busan']
})

data_city = data[['city']]

onehotencoder = OneHotEncoder(sparse_output = False).set_output(transform = 'pandas')

ont_hot_city = onehotencoder.fit_transform(data_city)

print(ont_hot_city)

In [ ]:
#3 다음 DataFrame에서 'height' 변수를 표준화(Standardization)한 후, 해당 변수의 최대값과 최소값의 차이를 출력하시오.

from sklearn.preprocessing import StandardScaler

data = pd.DataFrame({
'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
'height': [165, 170, 175, 180, 185]
})

data_num = data.select_dtypes('number')

std_scaler = StandardScaler()
data_num = std_scaler.fit_transform(data_num)

print(data_num.max() - data_num.min())

##### ❌4번

In [ ]:
#4 다음 데이터에서 'city' 변수의 결측치를 최빈값으로 대치한 후, 각 도시별 인원 수를 출력하시오.

from sklearn.impute import SimpleImputer

data = pd.DataFrame({
'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank'],
'city': ['Seoul', np.nan, 'Busan', 'Seoul', np.nan, 'Busan']
})

simpleimputer = SimpleImputer(strategy = 'most_frequent')

data[['city']] = simpleimputer.fit_transform(data[['city']])
print(data['city'].value_counts())

##### 5번

In [ ]:
#5 다음 데이터에서 sales 변수의 이상치를 상자그림 기준으로 제거한 후, 남은 데이터의 평균을 계산하여 출력하시오.

data = pd.DataFrame({
'product': ['A', 'B', 'C', 'D', 'E', 'F', 'G'],
'sales': [100, 120, 130, 400, 110, 115, 500]
})

Q1 = np.quantile(data['sales'], 0.25)
Q3 = np.quantile(data['sales'], 0.75)

IQR = Q3 - Q1

data = data.loc[ ( Q3 + IQR * 1.5 > data['sales'] ) & ( data['sales'] > Q1 - IQR * 1.5 ) ]

print(data['sales'].mean())

##### ❌6번

In [ ]:
#6 다음 데이터에서 score 변수를 구간의 폭이 동일하도록 4개의 구간으로 이산화(discretization)하고, 각 구간의 데이터 개수를 출력하시오.

from sklearn.preprocessing import KBinsDiscretizer

data = pd.DataFrame({
'name': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
'score': [55, 60, 65, 70, 75, 80, 85, 90, 95]
})

discretizer = KBinsDiscretizer(n_bins = 4, strategy = 'uniform', encode='ordinal').set_output(transform = 'pandas')
data[['score']] = discretizer.fit_transform(data[['score']])

print(data['score'].value_counts())

##### ❌7번

In [ ]:
#7 다음 데이터에서 math, english, science 3개 변수를 대상으로 주성분 분석(PCA)을 수행하여, 전체 분산의 80% 이상을 설명하는 최소 주성분 개수를 출력하시오.
# (단, 수치형 변수만 선택하고, 주성분 분석 전 표준화를 적용)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

data = pd.DataFrame({
'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
'math': [80, 85, 78, 92, 88],
'english': [75, 90, 85, 95, 92],
'science': [82, 88, 79, 94, 90]
})

std_scaler = StandardScaler()
data_num = data.select_dtypes('number')
data_num_standard = std_scaler.fit_transform(data_num)

pca = PCA(n_components = 0.8, svd_solver = 'full')

data_num_standard_pca = pca.fit_transform(data_num_standard)
print(pca.explained_variance_ratio_)
print(pca.n_components_)
print(data_num_standard_pca.shape[1])